### Climate Change Assignment in Python


In [2]:
import pandas as pd
import matplotlib 
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
%matplotlib inline  

#read in the climate data
climate=pd.read_csv('climate_change.csv')
climate.columns=['Year','Month','MEI','CO2','CH4','N2O','CFC11','CFC12','TSI','Aerosols', 'Temp']
climate.head()

,Year,Month,MEI,CO2,CH4,N2O,CFC11,CFC12,TSI,Aerosols,Temp
0,1983,5,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,0.109
1,1983,6,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,0.118
2,1983,7,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,0.137
3,1983,8,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,0.176
4,1983,9,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,0.149


In [3]:
train=climate[climate.Year<=2006]
test=climate[climate.Year>2006]

### Using sklearn

In [23]:
##import sklearn
from sklearn import linear_model
features=['MEI','CO2','CH4','N2O','CFC11','CFC12','TSI','Aerosols']
input_features=train[features]
#input_features.head()
output_feature=train['Temp']
LR_1=linear_model.LinearRegression()
LR_1.fit(input_features,output_feature)
print "Intercept: ", LR_1.intercept_
zip(input_features,LR_1.coef_)

Intercept:  -124.594260401


[('MEI', 0.06420531336752576),
 ('CO2', 0.0064573592723366378),
 ('CH4', 0.00012404189575251033),
 ('N2O', -0.01652800325747477),
 ('CFC11', -0.0066304888893799103),
 ('CFC12', 0.0038081032430232562),
 ('TSI', 0.093141083484999818),
 ('Aerosols', -1.5376132381050902)]

In [26]:
LR_1.fit
##The R2 value is 
LR_1.score(input_features,output_feature)

0.75089327705234177

In [30]:
from sklearn import feature_selection
LR_1.fit.feature_selection(input_features,output_feature)


AttributeError: 'function' object has no attribute 'feature_selection'

In [ ]:
##To get the e p-value (significance) of each coefficient?
class LinearRegression(linear_model.LinearRegression):

    def __init__(self,*args,**kwargs):
        # *args is the list of arguments that might go into the LinearRegression object
        # that we don't know about and don't want to have to deal with. Similarly, **kwargs
        # is a dictionary of key words and values that might also need to go into the orginal
        # LinearRegression object. We put *args and **kwargs so that we don't have to look
        # these up and write them down explicitly here. Nice and easy.

        if not "fit_intercept" in kwargs:
            kwargs['fit_intercept'] = False

        super(LinearRegression,self).__init__(*args,**kwargs)

    # Adding in t-statistics for the coefficients.
    def fit(self,x,y):
        # This takes in numpy arrays (not matrices). Also assumes you are leaving out the column
        # of constants.

        # Not totally sure what 'super' does here and why you redefine self...
        self = super(LinearRegression, self).fit(x,y)
        n, k = x.shape
        yHat = np.matrix(self.predict(x)).T

        # Change X and Y into numpy matricies. x also has a column of ones added to it.
        x = np.hstack((np.ones((n,1)),np.matrix(x)))
        y = np.matrix(y).T

        # Degrees of freedom.
        df = float(n-k-1)

        # Sample variance.     
        sse = np.sum(np.square(yHat - y),axis=0)
        self.sampleVariance = sse/df

        # Sample variance for x.
        self.sampleVarianceX = x.T*x

        # Covariance Matrix = [(s^2)(X'X)^-1]^0.5. (sqrtm = matrix square root.  ugly)
        self.covarianceMatrix = sc.linalg.sqrtm(self.sampleVariance[0,0]*self.sampleVarianceX.I)

        # Standard erros for the difference coefficients: the diagonal elements of the covariance matrix.
        self.se = self.covarianceMatrix.diagonal()[1:]

        # T statistic for each beta.
        self.betasTStat = np.zeros(len(self.se))
        for i in xrange(len(self.se)):
            self.betasTStat[i] = self.coef_[0,i]/self.se[i]

        # P-value for each beta. This is a two sided t-test, since the betas can be 
        # positive or negative.
        self.betasPValue = 1 - t.cdf(abs(self.betasTStat),df)
shareimprove this answer

### Using Stats models -which is not as complete as scikit-learn but has a syntax similar to R

In [35]:
train.N2O
model1=smf.ols(formula='Temp ~ MEI + CO2 + CH4+ N2O + CFC11+ CFC12+TSI + Aerosols',data=train)
Temp_fitted1=model1.fit()
Temp_fitted1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Temp   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.744
Method:                 Least Squares   F-statistic:                     103.6
Date:                Sat, 03 Oct 2015   Prob (F-statistic):           1.94e-78
Time:                        11:05:16   Log-Likelihood:                 280.10
No. Observations:                 284   AIC:                            -542.2
Df Residuals:                     275   BIC:                            -509.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -124.5943     19.887     -6.265      0.000      -163.744   -85.445
MEI            0.0642      0.006      9.923      0.000         0.051     0.077
CO2            0.0065      0.002      2.826      0.005         0.002     0.011
CH4            0.0001      0.001      0.240      0.810        -0.001     0.001
N2O           -0.0165      0.009     -1.930      0.055        -0.033     0.000
CFC11         -0.0066      0.002     -4.078      0.000        -0.010    -0.003
CFC12          0.0038      0.001      3.757      0.000         0.002     0.006
TSI            0.0931      0.015      6.313      0.000         0.064     0.122
Aerosols      -1.5376      0.213     -7.210      0.000        -1.957    -1.118
==============================================================================
Omnibus:                        8.740   Durbin-Watson:                   0.956
Prob(Omnibus):                  0.013   Jarque-Bera (JB):               10.327
Skew:                           0.289   Prob(JB):                      0.00572
Kurtosis:                       3.733   Cond. No.                     8.53e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.53e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

####1.1Build a linear regression model using all of the independant variables except Year and Month. Whta is the R-squared?

0.751

####1.2 Which variables are significant (p<0.005)

Significant variables:
* MEI 
* CO2
* CFC-11
* CFC-12
* TSI
* Aerosles

#### Current scientific opinion is that NO2 and CFC-11 are greenhouse gases.However their reegression coefficient atre negative. What does thsi likely signify?

In [36]:
train.corr()

,Year,Month,MEI,CO2,CH4,N2O,CFC11,CFC12,TSI,Aerosols,Temp
Year,1.000000,-0.027942,-0.036988,0.982749,0.915659,0.993845,0.569106,0.897012,0.170302,-0.345247,0.786797
Month,-0.027942,1.000000,0.000885,-0.106732,0.018569,0.013632,-0.013111,0.000675,-0.034606,0.014890,-0.099857
MEI,-0.036988,0.000885,1.000000,-0.041147,-0.033419,-0.050820,0.069000,0.008286,-0.154492,0.340238,0.172471
CO2,0.982749,-0.106732,-0.041147,1.000000,0.877280,0.976720,0.514060,0.852690,0.177429,-0.356155,0.788529
CH4,0.915659,0.018569,-0.033419,0.877280,1.000000,0.899839,0.779904,0.963616,0.245528,-0.267809,0.703255
N2O,0.993845,0.013632,-0.050820,0.976720,0.899839,1.000000,0.522477,0.867931,0.199757,-0.337055,0.778639
CFC11,0.569106,-0.013111,0.069000,0.514060,0.779904,0.522477,1.000000,0.868985,0.272046,-0.043921,0.407710
CFC12,0.897012,0.000675,0.008286,0.852690,0.963616,0.867931,0.868985,1.000000,0.255303,-0.225131,0.687558
TSI,0.170302,-0.034606,-0.154492,0.177429,0.245528,0.199757,0.272046,0.255303,1.000000,0.052117,0.243383
Aerosols,-0.345247,0.014890,0.340238,-0.356155,-0.267809,-0.337055,-0.043921,-0.225131,0.052117,1.000000,-0.384914


#### 2.2 Compute correlations between all variables in the tarining set. WHich of the independant variables is N2O highly crrelated with? (>0.7)

N2O is Highly correlated with:
    * CO2
    * CH4
    * CFC12
    

#### which of the variables in CFC-11 correlated with?

* CH4
* CFC-12


####3. Simplyfying the model. Given that the correlatiosn are so high, let us try to build a modle with just N2O,MEI, Aerosols and TSI. What is the coefficient of N2O in this model? How does this compare wiht the coeffient in the earlier model created with all variables? Enter the model R-squared. 

In [40]:
model2=smf.ols(formula='Temp~ N2O+ MEI+TSI+Aerosols',data=train)

In [39]:
Temp_fitted2=model2.fit()
Temp_fitted2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Temp   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.722
Method:                 Least Squares   F-statistic:                     184.9
Date:                Sat, 03 Oct 2015   Prob (F-statistic):           3.52e-77
Time:                        14:12:59   Log-Likelihood:                 266.64
No. Observations:                 284   AIC:                            -523.3
Df Residuals:                     279   BIC:                            -505.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -116.2269     20.223     -5.747      0.000      -156.036   -76.418
N2O            0.0253      0.001     19.307      0.000         0.023     0.028
MEI            0.0642      0.007      9.649      0.000         0.051     0.077
TSI            0.0795      0.015      5.344      0.000         0.050     0.109
Aerosols      -1.7017      0.218     -7.806      0.000        -2.131    -1.273
==============================================================================
Omnibus:                       10.908   Durbin-Watson:                   0.842
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               15.097
Skew:                           0.289   Prob(JB):                     0.000527
Kurtosis:                       3.971   Cond. No.                     5.00e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  5e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

####4 Automatically building the model: As we have seen  dropping some of the varibles form the model doesnt decrease by much. R has a step wise linear regression function; that python doesnt , but here is a piece of cod eI found from http://planspace.org/20150423-forward_selection_with_statsmodels/ that seems to do the trick!!! What is the R-squared value of the model produced. Which variable if any are eliminated?

In [73]:
##First remove Year and Month from the training et:
train=train.drop(['Year','Month'],axis=1)
test=train.drop(['Year','Month'],axis=1)

In [74]:
train.head()

,MEI,CO2,CH4,N2O,CFC11,CFC12,TSI,Aerosols,Temp
0,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,0.109
1,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,0.118
2,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,0.137
3,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,0.176
4,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,0.149


In [75]:
model_auto = forward_selected(train, 'Temp')

In [79]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Temp   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     118.8
Date:                Sat, 03 Oct 2015   Prob (F-statistic):           1.77e-79
Time:                        15:36:04   Log-Likelihood:                 280.07
No. Observations:                 284   AIC:                            -544.1
Df Residuals:                     276   BIC:                            -515.0
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -124.5152     19.850     -6.273      0.000      -163.592   -85.438
CO2            0.0064      0.002      2.821      0.005         0.002     0.011
MEI            0.0641      0.006      9.958      0.000         0.051     0.077
Aerosols      -1.5402      0.213     -7.244      0.000        -1.959    -1.122
TSI            0.0931      0.015      6.322      0.000         0.064     0.122
N2O           -0.0160      0.008     -1.933      0.054        -0.032     0.000
CFC11         -0.0066      0.002     -4.078      0.000        -0.010    -0.003
CFC12          0.0039      0.001      3.942      0.000         0.002     0.006
==============================================================================
Omnibus:                        8.543   Durbin-Watson:                   0.952
Prob(Omnibus):                  0.014   Jarque-Bera (JB):                9.913
Skew:                           0.291   Prob(JB):                      0.00704
Kurtosis:                       3.707   Cond. No.                     5.66e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.66e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [81]:
model_auto.rsquared 

0.7508408962216917

####5. How does this model hold to unseen data? wjhat is the R-squared?

In [87]:
test=climate[climate.Year>2006]
test=test.drop(['Year','Month'],axis=1)

In [92]:
test_predict=model_auto.predict(test)

In [104]:
#R2= 1-SSE/SST 
R2=1.0 - (((test.Temp -test_predict)**2).sum()/(((train.Temp.mean()- train)**2).sum()))

NameError: name 'train_predict' is not defined

In [102]:
R2

-2.6035126319811455